In [15]:
import matplotlib.pyplot as plt
import numpy as np
import math
import time
import scipy.signal
from scipy.optimize import curve_fit
import scipy.stats as stats
import pandas as pd
import os
from  scipy.stats import chi2_contingency
#from statsmodels.stats.weightstats import DescrStatsW
import glob
import csv
import fileinput
from glob import glob
import re
from scipy.signal import butter, filtfilt
import scipy.integrate as spi
from scipy.stats import chi2
from scipy.signal import find_peaks
from scipy.interpolate import interp1d

from scipy import integrate
try:
    from scipy.integrate import trapezoid as trapz
except ImportError:
    from scipy.integrate import trapz


#ej de path=r'C:\Users\Sergio\Desktop\L6y7\18-04-23'
#path='C:/Users/Nicolás Molina/Desktop/L6-7/04-05-23-14kV'
#path=r'C:\Users\Sergio\Desktop\L6y7\04-05-23-19kV'

path=r'C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\caracterizacion gan'

#get_ipython().run_line_magic('matplotlib', 'inline')
get_ipython().run_line_magic('matplotlib', 'qt5')

import matplotlib.ticker as ticker


In [16]:
#plt.close("all")
path=r"C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\caracterizacion gan"  #'C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\cef3 solo'
plt.close("all")
os.chdir (path)

try:
    from scipy.integrate import trapezoid as trapz
except ImportError:
    from scipy.integrate import trapz

from matplotlib.ticker import LogLocator, FuncFormatter

## plots crudos
intlamp=np.loadtxt("curva lampara con diodo-corregida.txt",delimiter="	",skiprows=1,unpack=True)
intdiodo=np.loadtxt("curva diodo.txt",delimiter=",",skiprows=1,unpack=True)
transistor=np.loadtxt("respuesta transistor.txt",delimiter="	",skiprows=1,unpack=True)

espectrolamp=np.loadtxt("lampara medida de fabrica.txt",delimiter=",",skiprows=1,unpack=True)

lambdaespec=espectrolamp[0]
irradiazaespeclamp=espectrolamp[1]

lambdalamp=intlamp[0]
escala=intlamp[1]
porcentaje=intlamp[2]
intlamp=escala*porcentaje

lambdadiodo=intdiodo[0]
responsdiodo=intdiodo[1]

lambdagan=transistor[0]
respuestagan=transistor[1]
##########
"""
plt.figure()

plt.plot(lambdalamp,escala*porcentaje) 
plt.title("Respuesta del diodo al setup")   
plt.xlabel('$\lambda$ (nm)')
plt.ylabel('Corriente del diodo (pA)')
plt.legend()
plt.tight_layout()
plt.grid()

plt.figure()
plt.plot(lambdadiodo,responsdiodo)
plt.title("Responsividad del diodo (fabricante)")  
plt.xlabel('$\lambda$ (nm)')
plt.ylabel('Responsividad (A/W)')
plt.grid()
plt.tight_layout()
plt.legend()

plt.figure()
plt.plot(lambdagan,respuestagan) 
plt.title("Respuesta del transistor al setup (salto de corriente)")   
plt.xlabel('$\lambda$ (nm)')
plt.ylabel('Salto de corriente ($mA$)')
plt.grid()
plt.tight_layout()
plt.legend()
####################################
"""

# Crear un rango común de puntos para interpolar (puedes ajustar este rango según tus datos)
x_common = np.linspace(max(min(lambdalamp), min(lambdadiodo),min(lambdagan)), min(max(lambdalamp), max(lambdadiodo),max(lambdagan)), num=1000)

# Interpolar los datos
f1 = interp1d(lambdalamp, intlamp, kind='linear', fill_value="extrapolate")
f2 = interp1d(lambdadiodo, responsdiodo, kind='linear', fill_value="extrapolate")
f3= interp1d(lambdagan, respuestagan, kind='linear', fill_value="extrapolate")
# Evaluar las funciones interpoladas en los puntos comunes


y1_interp = f1(x_common)
y2_interp = f2(x_common)
y3_interp=f3(x_common)
################################################################################

errorlockin=0.01*y1_interp
errorsmu=0.003 #en mA !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# Multiplicar los valores interpolados
potencialamp = y1_interp*10**(-12) *1/ y2_interp  #hago pA / (A/W)


intensidadsetup=potencialamp/0.076


A_detector = 0.07  # 0.1 mm^2 

responsividadgan=y3_interp/(1000*intensidadsetup*A_detector)  #AGREGAR MULTIPLICAR EN LA DIVISION POR EL AREA DEL TRANSISTOR
# Guardar el resultado en un archivo si es necesario





np.savetxt('INTENSIDAD SETUP.txt', np.column_stack((x_common, intensidadsetup)))


errorintensidadsetup=intensidadsetup*np.sqrt((errorlockin/y1_interp)**2)

errorgan=errorsmu

errorresponsividadgan=responsividadgan*np.sqrt((errorintensidadsetup/intensidadsetup)**2+(errorsmu/y3_interp)**2)

##########################################################

# Graficar los resultados

plt.figure()
plt.plot(lambdaespec, irradiazaespeclamp,ls="-",linewidth=2)
plt.xlabel('$\lambda$ (nm)')
plt.ylabel('Irradianza espectral ($W/cm^2 \cdot nm$)')
plt.grid()
plt.legend()
#plt.title('Irradianza espectral de la lámpara')
plt.tight_layout()


plt.figure()
plt.plot(x_common, y1_interp,ls="-")
plt.fill_between(x_common, y1_interp - errorlockin, y1_interp + errorlockin, color="red", alpha=0.5, label="Error")
plt.xlabel('$\lambda$ (nm)')
plt.ylabel('Corriente en el diodo ($pA$)')
plt.grid()
plt.legend()
#plt.title('Respuesta del diodo al setup')
plt.tight_layout()


plt.figure()
#plt.plot(lambdadiodo,responsdiodo,label="Respuesta del diodo del fabricante",linewidth=2)
plt.plot(x_common, y2_interp,ls="-",linewidth=2)
plt.xlabel('$\lambda$ (nm)')
plt.ylabel('Responsividad del diodo ($A/W$)')
plt.grid()
plt.legend()
#plt.title('Responsividad del diodo (dato de fábrica)')
plt.tight_layout()

"""
plt.figure()
plt.plot(x_common, potencialamp, linestyle='--')
plt.legend()
plt.xlabel('$\lambda$ (nm)')
plt.ylabel('Potencia de la lámpara ($W$)')
plt.grid()
plt.title('Potencia de la lámpara medida en el diodo (0.076$mm^2$)')
plt.show()
"""
plt.figure()
plt.plot(x_common,y3_interp) 
plt.fill_between(x_common, y3_interp - errorgan, y3_interp + errorgan, color="red", alpha=0.5, label="Error")
#plt.title("Respuesta del transistor al setup")   
plt.xlabel('$\lambda$ (nm)')
plt.ylabel('Salto de corriente ($mA$)')
plt.grid()
plt.legend()
plt.tight_layout()

plt.figure()
plt.plot(x_common, intensidadsetup, linestyle='--')
plt.fill_between(x_common, intensidadsetup - errorintensidadsetup, intensidadsetup + errorintensidadsetup, color="red", alpha=0.5, label="Error")
plt.legend()
plt.xlabel('$\lambda$ (nm)')
plt.ylabel('Intensidad de la lámpara ($W/mm^2$)')
plt.grid()
#plt.title('Intensidad espectral del setup ')
plt.tight_layout()
plt.show()


plt.figure()
plt.plot(x_common, responsividadgan, linestyle='--')
plt.fill_between(x_common, responsividadgan - errorresponsividadgan, responsividadgan + errorresponsividadgan, color="red", alpha=0.5, label="Error")
plt.legend()
plt.xlabel('$\lambda$ (nm)')
plt.ylabel('Responsividad del transistor de GaN-AlGaN ($A/W$)')
#plt.yscale("log")
plt.grid()
#plt.title('Responsividad del transistor')
plt.tight_layout()
plt.show()





<>:34: SyntaxWarning: invalid escape sequence '\l'
<>:113: SyntaxWarning: invalid escape sequence '\l'
<>:114: SyntaxWarning: invalid escape sequence '\c'
<>:124: SyntaxWarning: invalid escape sequence '\l'
<>:135: SyntaxWarning: invalid escape sequence '\l'
<>:142: SyntaxWarning: invalid escape sequence '\l'
<>:156: SyntaxWarning: invalid escape sequence '\l'
<>:166: SyntaxWarning: invalid escape sequence '\l'
<>:178: SyntaxWarning: invalid escape sequence '\l'
<>:34: SyntaxWarning: invalid escape sequence '\l'
<>:113: SyntaxWarning: invalid escape sequence '\l'
<>:114: SyntaxWarning: invalid escape sequence '\c'
<>:124: SyntaxWarning: invalid escape sequence '\l'
<>:135: SyntaxWarning: invalid escape sequence '\l'
<>:142: SyntaxWarning: invalid escape sequence '\l'
<>:156: SyntaxWarning: invalid escape sequence '\l'
<>:166: SyntaxWarning: invalid escape sequence '\l'
<>:178: SyntaxWarning: invalid escape sequence '\l'
C:\Users\Nicolás Molina\AppData\Local\Temp\ipykernel_27388\1551355

## Condiciones de medida acelerador

In [17]:
path=r"C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\films hexagonales"  #'C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\cef3 solo'
#plt.close("all")
os.chdir (path)

try:
    from scipy.integrate import trapezoid as trapz
except ImportError:
    from scipy.integrate import trapz

Efoton=1.2398*10**(-3)*1/x_common   #energia foton en mev
Eprot=10 #energia proton en mev



Ip=(1)*10**(-9)   #1aca hay puestos 100pa A=C/s   


qprot=(1.6)*10**(-19)   #en coulombs C
tasaprot=Ip/qprot
Wprot=Eprot*(1.6)*10**(-13)*tasaprot   #pase la energia a Joules  
Ynf=2000    #rendimiento de centelleo en fotones/mev
Ypvt=Ynf*0.28127
Yps=Ynf*0.20316
Ypvtcef=Ynf*0.19886


d_centellador = 22  # Diámetro del centellador en mm
A_centellador = np.pi * (d_centellador / 2)**2  # Área del centellador en mm2

# Área del detector en cm^2
eta = A_detector / A_centellador  # Eficiencia de detección

## plots crudos
pl=np.loadtxt("hexagonales pl.txt",delimiter="	",skiprows=4,unpack=True)

titulos=["Ps/Nf 10%","PS","PVT","PVT/$CeF_{3}$ 10%","PVT/$CeF_{3}$ 10% (sep)"]
#for i in [1,3,4,5,6]:
lambdapl=pl[0]

plnf=pl[1]
plps=pl[3]
plpvt=pl[4]
plpvtcef=pl[5]



#PL NORMALIZADO e interpolado
fnf= interp1d(lambdapl, plnf, kind='linear', fill_value="extrapolate")
fps=interp1d(lambdapl, plps, kind='linear', fill_value="extrapolate")
fpvt=interp1d(lambdapl, plpvt, kind='linear', fill_value="extrapolate")
fpvtcef=interp1d(lambdapl, plpvtcef, kind='linear', fill_value="extrapolate")

ynf=fnf(x_common)/max(fnf(x_common))    
yps=fps(x_common)/max(fps(x_common))
ypvt=fpvt(x_common)/max(fpvt(x_common))
ypvtcef=fpvtcef(x_common)/max(fpvtcef(x_common))
#plt.figure()
#plt.plot(x_common,ynf)


areanf=trapz(fnf(x_common)/max(fnf(x_common)),x_common)
areaps=trapz(fps(x_common)/max(fnf(x_common)),x_common)
areapvt=trapz(fpvt(x_common)/max(fnf(x_common)),x_common)
areapvtcef=trapz(fpvtcef(x_common)/max(fnf(x_common)),x_common)

# integral del pl normalizado
Znf=trapz(ynf,x_common)
Zps=trapz(yps,x_common)
Zpvt=trapz(ypvt,x_common)
Zpvtcef=trapz(ypvtcef,x_common)

nf=ynf/Znf
ps=yps/Zps
pvt=ypvt/Zpvt
pvtcef=ypvtcef/Zpvtcef

print(areanf,"aaa",areapvt/areanf,"aaa",areaps/areanf,"aaa",areapvtcef/areanf)


integralps= trapz(Yps*Wprot*Efoton*responsividadgan*ps*eta,x_common)
integralnf= trapz(Ynf*Wprot*Efoton*responsividadgan*nf*eta,x_common)
integralpvt=trapz(Ypvt*Wprot*Efoton*responsividadgan*pvt*eta,x_common)
integralpvtcef=trapz(Ypvtcef*Wprot*Efoton*responsividadgan*pvtcef*eta,x_common)
# Ejemplo: 5% de error relativo en cada punto



print("salto de corriente nf=",integralnf,"+-", f"para {Ip*10**(9)} nA")
print("salto de corriente pvt=",integralpvt, "+-",f"para  {Ip*10**(9)} nA")
print("salto de corriente ps=",integralps, "+-",f"para {Ip*10**(9)} nA")
print("salto de corriente pvtcef=",integralpvtcef, "+-",f"para {Ip*10**(9)} nA")

print("por nanosegundo le llegan al centellador",tasaprot/(10**9),"protones de",Eprot,"MeV")


30.181606837448882 aaa 0.2812702590011842 aaa 0.20316890656713066 aaa 0.1988663530244489
salto de corriente nf= 0.009059084446690496 +- para 1.0 nA
salto de corriente pvt= 0.006696902508391507 +- para  1.0 nA
salto de corriente ps= 0.00524599471755977 +- para 1.0 nA
salto de corriente pvtcef= 0.0044444422224190535 +- para 1.0 nA
por nanosegundo le llegan al centellador 6.25 protones de 10 MeV


# FINAL PERO EL ESPECTRO ESTA FEO ACA

In [22]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate

try:
    from scipy.integrate import trapezoid as trapz
except ImportError:
    from scipy.integrate import trapz
from scipy.interpolate import interp1d
plt.close("all")

# Parámetros iniciales
Ynf = 2000  # Rendimiento de centelleo en fotones/mev
Ypvt = Ynf * 0.28127
Yps = Ynf * 0.20316
Ypvtcef = Ynf * 0.19886

d_centellador = 24  # Diámetro del centellador en mm
A_centellador = np.pi * (d_centellador / 2)**2  # Área del centellador en mm2
A_detector=0.07


Efoton=1.2398*10**(-3)*1/x_common 
# Área del detector en cm^2 (asumiendo que A_detector está definido)

eta = A_detector / A_centellador  # Eficiencia de detección
path=r"C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\films hexagonales" 
os.chdir(path)
# Cargar datos
pl = np.loadtxt("hexagonales pl.txt", delimiter="\t", skiprows=4, unpack=True)

lambdapl = pl[0]
plnf = pl[1]
plps = pl[3]
plpvt = pl[4]
plpvtcef = pl[5]

# Interpolación y normalización
x_common = np.linspace(min(lambdapl), max(lambdapl), 1000)  # Rango común para interpolación
fnf = interp1d(lambdapl, plnf, kind='linear', fill_value="extrapolate")
fps = interp1d(lambdapl, plps, kind='linear', fill_value="extrapolate")
fpvt = interp1d(lambdapl, plpvt, kind='linear', fill_value="extrapolate")
fpvtcef = interp1d(lambdapl, plpvtcef, kind='linear', fill_value="extrapolate")

ynf = fnf(x_common) / max(fnf(x_common))
yps = fps(x_common) / max(fps(x_common))
ypvt = fpvt(x_common) / max(fpvt(x_common))
ypvtcef = fpvtcef(x_common) / max(fpvtcef(x_common))

# Integral del PL normalizado
Znf = trapz(ynf, x_common)
Zps = trapz(yps, x_common)
Zpvt = trapz(ypvt, x_common)
Zpvtcef = trapz(ypvtcef, x_common)

nf = ynf / Znf
ps = yps / Zps
pvt = ypvt / Zpvt
pvtcef = ypvtcef / Zpvtcef

# Definir los rangos de Ip y Eprot
Ip_values = np.logspace(np.log10(0.00001e-9), np.log10(200e-9), 600)  # Corriente en escala log  #0.01e-9  y 100e-9
Eprot_values = np.logspace(np.log10(0.0001), np.log10(13), 200)  # Energía en escala log
Eprot_values = np.linspace(0,12,200)
# Constantes adicionales
qprot = 1.602176634e-19  # Carga del protón en Coulombs



# Inicializar listas para almacenar los resultados
Ip_results = []
Eprot_results = []
integralnf_results = []

# Bucle para variar Ip y Eprot
for Ip in Ip_values:
    for Eprot in Eprot_values:
        # Calcular tasaprot y Wprot
        tasaprot = Ip / qprot
        Wprot = Eprot * (1.6) * 10**(-13) * tasaprot
        
        # Calcular integralnf
        integralnf = trapz(Ynf * Wprot * Efoton * responsividadgan * nf * eta, x_common)
        
        # Guardar todos los resultados
        Ip_results.append(Ip)
        Eprot_results.append(Eprot)
        integralnf_results.append(integralnf)

# Convertir las listas a arrays de numpy para facilitar la manipulación
Ip_results = np.array(Ip_results)
Eprot_results = np.array(Eprot_results)
integralnf_results = np.array(integralnf_results)*1000

# Crear una máscara para los valores de integralnf no saturados
mask = integralnf_results <= 1.8

# Crear una máscara adicional para valores >= 0.0001
mask_color = integralnf_results >= 0.1

# Combinar ambas máscaras
mask_final = mask & mask_color

# Crear una paleta de colores personalizada
from matplotlib.colors import ListedColormap
cmap = plt.cm.viridis  # Paleta base
cmap.set_over('red')   # Color para valores mayores a 0.0018

# Graficar los resultados
plt.figure(figsize=(10, 8))

# Graficar todos los puntos
scatter = plt.scatter(Ip_results * 1e9, Eprot_results, c=integralnf_results, 
                      cmap=cmap, marker='o', vmax=1.8)  # vmax define el límite superior para la paleta base

# Añadir barra de colores
cbar = plt.colorbar(scatter, extend='max')  # 'extend' añade una flecha para indicar valores superiores a vmax
cbar.set_label('Fotocorriente (mA)', fontsize=14)
cbar.ax.tick_params(labelsize=12)
cbar.ax.text(2, 1.02, 'Saturación', transform=cbar.ax.transAxes, 
             fontsize=12, color='red', va='bottom', ha='center')  # Etiqueta de saturación

# Añadir rectángulos en las coordenadas especificadas
rect1 = plt.Rectangle((0.09, 9.8), 0.02, 0.4, linewidth=2, edgecolor='red', facecolor='none')  # Rectángulo rojo en (0.1 nA, 10 MeV)
rect2 = plt.Rectangle((0.9, 9.8), 0.2, 0.4, linewidth=2, edgecolor='blue', facecolor='none')  # Rectángulo azul en (1 nA, 10 MeV)

# Añadir los rectángulos al gráfico
plt.gca().add_patch(rect1)
plt.gca().add_patch(rect2)

# Etiquetas y título
plt.xlabel('Corriente de protones (nA)', fontsize=14)
plt.ylabel('Energía de protones (MeV)', fontsize=14)
plt.xscale("log")
#plt.xlim(13)
plt.grid(True)
plt.tick_params(axis='both', which='major', labelsize=12)
plt.tight_layout()
plt.show()

# Calcular el flujo para los valores no saturados y dentro del rango de color
flujo = Ip_results[mask_final] / (A_centellador / 1000 * np.pi * Eprot_results[mask_final])  # Flujo = Corriente / (Área * Energía)

# Graficar Energía vs Flujo para valores no saturados y saturados
plt.figure(figsize=(10, 8))

# Graficar puntos no saturados (fotocorriente <= 0.0018)
plt.scatter(Eprot_results[mask_final], flujo, c=integralnf_results[mask_final], 
            cmap='viridis', marker='o', vmin=0.1, vmax=1.8)

# Graficar puntos saturados (fotocorriente > 0.0018) en rojo
mask_saturados = integralnf_results > 1.8
flujo_saturados = Ip_results[mask_saturados] / (A_centellador / 1000 * np.pi * Eprot_results[mask_saturados])
plt.scatter(Eprot_results[mask_saturados], flujo_saturados, c='red', marker='o', 
            label='Saturación')

# Añadir barra de colores
cbar = plt.colorbar()
cbar.set_label('Fotocorriente (mA)', fontsize=14)
cbar.ax.tick_params(labelsize=12)

# Etiquetas y título
plt.xlabel('Energía de protones (MeV)', fontsize=14)
plt.ylabel('Flujo de $H^+$ (partículas $cm^{-2} sr^{-1} s^{-1} MeV^{-1}$)', fontsize=14)
plt.xscale("log")
plt.yscale("log")
plt.grid(True)

plt.tick_params(axis='both', which='major', labelsize=12)
plt.legend(fontsize=12)  # Añadir leyenda
plt.tight_layout()
plt.show()

# SRIM GAN

In [19]:
#plt.close("all")
path=r"C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\srim ganalgan"  #'C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\cef3 solo'
plt.close("all")
os.chdir (path)

from matplotlib.ticker import LogLocator, FuncFormatter

## plots crudos
ioniz=np.loadtxt("ioniz.txt",delimiter=",",skiprows=45,unpack=True)

depth=ioniz[0]*10 #paso a nm
energy=ioniz[1]/10  # en ev/nm

plt.figure()
#plt.axvline(,c="red",linestyle="--")
plt.plot(depth,energy)
plt.xscale("log")
plt.grid()




# Espectro

In [20]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import trapz
from scipy.interpolate import interp1d
plt.close("all")

# Parámetros iniciales
Ynf = 2000  # Rendimiento de centelleo en fotones/mev
Ypvt = Ynf * 0.28127
Yps = Ynf * 0.20316
Ypvtcef = Ynf * 0.19886

d_centellador = 24  # Diámetro del centellador en mm
A_centellador = np.pi * (d_centellador / 2)**2  # Área del centellador en mm2
Efoton=1.2398*10**(-3)*1/x_common 
# Área del detector en cm^2 (asumiendo que A_detector está definido)

eta = A_detector / A_centellador  # Eficiencia de detección
path = r"C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\films hexagonales" 
os.chdir(path)
# Cargar datos
pl = np.loadtxt("hexagonales pl.txt", delimiter="\t", skiprows=4, unpack=True)

lambdapl = pl[0]
plnf = pl[1]
plps = pl[3]
plpvt = pl[4]
plpvtcef = pl[5]

# Interpolación y normalización
x_common = np.linspace(min(lambdapl), max(lambdapl), 1000)  # Rango común para interpolación
fnf = interp1d(lambdapl, plnf, kind='linear', fill_value="extrapolate")
fps = interp1d(lambdapl, plps, kind='linear', fill_value="extrapolate")
fpvt = interp1d(lambdapl, plpvt, kind='linear', fill_value="extrapolate")
fpvtcef = interp1d(lambdapl, plpvtcef, kind='linear', fill_value="extrapolate")

ynf = fnf(x_common) / max(fnf(x_common))
yps = fps(x_common) / max(fps(x_common))
ypvt = fpvt(x_common) / max(fpvt(x_common))
ypvtcef = fpvtcef(x_common) / max(fpvtcef(x_common))

# Integral del PL normalizado
Znf = trapz(ynf, x_common)
Zps = trapz(yps, x_common)
Zpvt = trapz(ypvt, x_common)
Zpvtcef = trapz(ypvtcef, x_common)

nf = ynf / Znf
ps = yps / Zps
pvt = ypvt / Zpvt
pvtcef = ypvtcef / Zpvtcef

# Definir los rangos de Ip y Eprot
Ip_values = np.logspace(np.log10(0.001e-9), np.log10(1000000e-9), 900)  # Corriente en escala log
Eprot_values = np.logspace(np.log10(0.0001), np.log10(13), 400)  # Energía en escala log

# Constantes adicionales
qprot = 1.602176634e-19  # Carga del protón en Coulombs


# Inicializar listas para almacenar los resultados
Ip_results = []
Eprot_results = []
integralnf_results = []

# Bucle para variar Ip y Eprot
for Ip in Ip_values:
    for Eprot in Eprot_values:
        # Calcular tasaprot y Wprot
        tasaprot = Ip / qprot
        Wprot = Eprot * (1.6) * 10**(-13) * tasaprot
        
        # Calcular integralnf
        integralnf = trapz(Ypvtcef * Wprot * Efoton * responsividadgan * pvtcef * eta, x_common)
        
        # Guardar todos los resultados
        Ip_results.append(Ip)
        Eprot_results.append(Eprot)
        integralnf_results.append(integralnf)

# Convertir las listas a arrays de numpy para facilitar la manipulación
Ip_results = np.array(Ip_results)
Eprot_results = np.array(Eprot_results)
integralnf_results = np.array(integralnf_results)

# Crear una máscara para los valores de integralnf no saturados
mask = integralnf_results <= 0.0018

# Crear una máscara adicional para valores >= 0.0001
mask_color = integralnf_results >= 0.0001

# Combinar ambas máscaras
mask_final = mask & mask_color

# Crear una máscara para valores menores a 0.0001
mask_menor_0001 = integralnf_results < 0.0001

# Crear una paleta de colores personalizada
from matplotlib.colors import ListedColormap
cmap = plt.cm.viridis  # Paleta base
cmap.set_over('red')   # Color para valores mayores a 0.0018
"""
# Graficar los resultados
plt.figure(figsize=(10, 8))

# Graficar todos los puntos
scatter = plt.scatter(Ip_results * 1e9, Eprot_results, c=integralnf_results, 
                      cmap=cmap, marker='o', vmax=0.0018)  # vmax define el límite superior para la paleta base

# Graficar puntos con valores menores a 0.0001 con un patrón de líneas a 45 grados
plt.scatter(Ip_results[mask_menor_0001] * 1e9, Eprot_results[mask_menor_0001], 
            c='none', edgecolor='black', marker='o', hatch='////', 
            label='Sin detección (< 0.0001 A)')

# Añadir barra de colores
cbar = plt.colorbar(scatter, extend='max')  # 'extend' añade una flecha para indicar valores superiores a vmax
cbar.set_label('Fotocorriente (A)', fontsize=14)
cbar.ax.tick_params(labelsize=12)
cbar.ax.text(2, 1.02, 'Saturación', transform=cbar.ax.transAxes, 
             fontsize=12, color='red', va='bottom', ha='center')  # Etiqueta de saturación

# Añadir rectángulos en las coordenadas especificadas
rect1 = plt.Rectangle((0.09, 9.8), 0.02, 0.4, linewidth=2, edgecolor='red', facecolor='none')  # Rectángulo rojo en (0.1 nA, 10 MeV)
rect2 = plt.Rectangle((0.9, 9.8), 0.2, 0.4, linewidth=2, edgecolor='blue', facecolor='none')  # Rectángulo azul en (1 nA, 10 MeV)

# Añadir los rectángulos al gráfico
plt.gca().add_patch(rect1)
plt.gca().add_patch(rect2)

# Etiquetas y título
plt.xlabel('Corriente de protones (nA)', fontsize=14)
plt.ylabel('Energía de protones (MeV)', fontsize=14)
plt.xscale("log")
plt.grid(True)
plt.tick_params(axis='both', which='major', labelsize=12)
plt.legend(fontsize=12)  # Añadir leyenda
plt.tight_layout()
plt.show()

"""

# Definir límites de detección
limite_inferior = 0.0001
limite_superior = 0.0018

# Calcular el flujo
flujo = Ip_results / (A_centellador / 1000 * np.pi * Eprot_results)  # Flujo = Corriente / (Área * Energía)



# Máscaras para valores detectables, no detectados y saturados
mask_plot = (integralnf_results <= limite_superior) & (integralnf_results >= limite_inferior)



mask_no_detecta = integralnf_results < limite_inferior
mask_saturacion = integralnf_results > limite_superior

# Crear la figura
plt.figure(figsize=(12, 8))

# Graficar solo los puntos detectables
scatter = plt.scatter(Eprot_results[mask_plot], flujo[mask_plot], 
                      c=integralnf_results[mask_plot], cmap='viridis', marker='o', 
                      vmin=limite_inferior, vmax=limite_superior)

# Ordenar valores de Eprot_results para que fill_between funcione correctamente
sorted_indices = np.argsort(Eprot_results)
Eprot_sorted = Eprot_results[sorted_indices]
flujo_sorted = flujo[sorted_indices]
integralnf_sorted = integralnf_results[sorted_indices]

# Colorear el área de no detección (debajo de la curva)
plt.fill_between(Eprot_sorted, flujo_sorted, np.min(flujo_sorted), 
                 where=integralnf_sorted < limite_inferior, color='gray', alpha=1, label='No detección')

# Colorear el área de saturación (por encima de la curva)
plt.fill_between(Eprot_sorted, flujo_sorted, np.max(flujo_sorted), 
                 where=integralnf_sorted > limite_superior, color='red', alpha=1, label='Saturación')

# Barra de colores
cbar = plt.colorbar(scatter)
cbar.set_label('Fotocorriente (A)', fontsize=14)
cbar.ax.tick_params(labelsize=12)

# Etiquetas y título
plt.xlabel('Energía de protones (MeV)', fontsize=14)
plt.ylabel('Flujo de $H^+$ (partículas $cm^{-2} sr^{-1} s^{-1} MeV^{-1}$)', fontsize=14)
plt.xscale("log")
plt.yscale("log")

#plt.grid(True, which="both", linestyle="--", linewidth=0.5)
plt.tick_params(axis='both', which='major', labelsize=12)
plt.legend(fontsize=12, loc='upper right')  # Añadir leyenda
plt.tight_layout()
plt.show()


ImportError: cannot import name 'trapz' from 'scipy.integrate' (c:\Users\Nicolás Molina\AppData\Local\Programs\Python\Python313\Lib\site-packages\scipy\integrate\__init__.py)

# dosis grays

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import trapz
from matplotlib.colors import LogNorm

plt.close("all")

# Masa del cuerpo humano en kg (ajustable)
masa = 0.001  # kg

# Valores de Ip y Eprot en escala logarítmica
Ip_values = np.logspace(np.log10(1e-18), np.log10(1e18), 900)  # Corriente (A)
Eprot_values = np.logspace(np.log10(0.0001), np.log10(1e7), 400)  # Energía (MeV)

# Constantes adicionales
qprot = 1.602176634e-19  # Carga del protón en Coulombs

# Inicializar listas para almacenar los resultados
Ip_results = []
Eprot_results = []
integralnf_results = []
flujo_results = []

# Bucle para variar Ip y Eprot
for Ip in Ip_values:
    for Eprot in Eprot_values:
        # Calcular tasaprot y Wprot
        tasaprot = Ip / qprot
        Wprot = Eprot * (1.6) * 10**(-13) * tasaprot

        # Calcular integralnf
        integralnf = trapz(Ypvtcef * Wprot * Efoton * responsividadgan * pvtcef * eta, x_common)

        # Calcular flujo
        

        # Guardar todos los resultados
        Ip_results.append(Ip)
        Eprot_results.append(Eprot)
        integralnf_results.append(integralnf)
        

# Convertir las listas a arrays de numpy para facilitar la manipulación
Ip_results = np.array(Ip_results)
Eprot_results = np.array(Eprot_results)
integralnf_results = np.array(integralnf_results)
flujo = Ip_results / (A_centellador / 1000 * np.pi * Eprot_results)  # Flujo = Corriente / (Área * Energía)

# Calcular la dosis en Gray/segundo
energia_depositada = integralnf_results * Eprot_results * 1.60218e-13 / qprot  # J/s
dosis_gy_s = energia_depositada / masa

# Crear la figura
plt.figure(figsize=(12, 8))

# Graficar todos los puntos sin máscara
scatter = plt.scatter(Eprot_results, flujo, 
                      c=dosis_gy_s, cmap='viridis', marker='o', 
                     norm=LogNorm(vmin=1e-12, vmax=1e10))  # Escala logarítmica

# Barra de colores
cbar = plt.colorbar(scatter)
cbar.set_label('Dosis (Gy/s)', fontsize=14)
cbar.ax.tick_params(labelsize=12)
cbar.mappable.set_clim(0.0000000001, 10e10)  # Limitar colorbar de 0 a 10^10

# Etiquetas y título
plt.xlabel('Energía de protones (MeV)', fontsize=14)
plt.ylabel('Flujo de $H^+$ (partículas $cm^{-2} sr^{-1} s^{-1} MeV^{-1}$)', fontsize=14)
plt.xscale("log")
plt.yscale("log")
plt.ylim(1e-12, 1e10)  # Fijar los límites del eje y
plt.tick_params(axis='both', which='major', labelsize=12)
plt.legend(fontsize=12, loc='upper right')  # Añadir leyenda
plt.tight_layout()
plt.show()


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# Masa del cuerpo humano en kg (ajustable)
masa = 0.001  # kg

# Valores de Ip y Eprot en escala logarítmica
Ip_values = np.logspace(np.log10(1e-20), np.log10(1e18), 1000)  # Corriente (A)
Eprot_values = np.logspace(np.log10(0.0001), np.log10(16), 300)  # Energía (MeV)

# Constantes
qprot = 1.602176634e-19  # Carga del protón en Coulombs
A_centellador = 1.0  # Definir área del centellador en cm² (AJUSTAR SEGÚN NECESARIO)

# Inicializar listas para almacenar los resultados
Ip_results = []
Eprot_results = []
flujo_results = []

# Bucle para variar Ip y Eprot
for Ip in Ip_values:
    for Eprot in Eprot_values:
        flujo = Ip / (A_centellador / 1000 * np.pi * Eprot)  # Flujo corregido

        # Guardar los resultados
        Ip_results.append(Ip)
        Eprot_results.append(Eprot)
        flujo_results.append(flujo)

# Convertir listas a arrays de NumPy
Ip_results = np.array(Ip_results)
flujo_results = np.array(flujo_results)
Eprot_results = np.array(Eprot_results)

# Calcular la dosis en Gray/segundo
energia_depositada = Ip_results * Eprot_results * 1.60218e-13 / qprot  # J/s
dosis_gy_s = energia_depositada / masa

# Definir el umbral para destrucción instantánea
umbral_destruccion = 1e5  # 100000 Gy/s

# Crear el mapa de colores y la normalización logarítmica
cmap = plt.get_cmap('viridis')
norm = LogNorm(vmin=1e-12, vmax=umbral_destruccion)

# Crear máscara para valores superiores al umbral
mascara_destruccion = dosis_gy_s > umbral_destruccion

# Crear la figura
plt.figure(figsize=(12, 8))

# Graficar los puntos con escala logarítmica
scatter = plt.scatter(Eprot_results, flujo_results, c=dosis_gy_s, cmap=cmap, norm=norm, marker='o')

# Crear la `colorbar` correctamente
cbar = plt.colorbar(scatter)
cbar.set_label('Dosis (Gy/s)', fontsize=14)
cbar.ax.tick_params(labelsize=12)

# Agregar puntos de destrucción instantánea en rojo
plt.scatter(Eprot_results[mascara_destruccion], flujo_results[mascara_destruccion], 
            color='red', marker='o', label="Destrucción a los pocos segundos")

# Configurar escala logarítmica en los ejes
plt.xscale("log")
plt.yscale("log")
plt.ylim(1e-12, 1e10)

# Etiquetas
plt.xlabel('Energía de protones (MeV)', fontsize=14)
plt.ylabel('Flujo de $H^+$ (partículas $cm^{-2} sr^{-1} s^{-1} MeV^{-1}$)', fontsize=14)
plt.tick_params(axis='both', which='major', labelsize=12)

# Agregar leyenda
plt.legend(fontsize=12, loc='upper right')

plt.tight_layout()
plt.show()
